In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os


username = "fake"
password = "password"  
host = 'localhost'  
port = '3306'       
database = 'football_db'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

tables = engine.table_names()

player_tables = [table for table in tables if "player" in table]
dfs = {}

for table in player_tables:
    df = pd.read_sql_table(table, con=engine)
    dfs[table] = df
    print(f"Loaded DataFrame for table: {table}")

C:\Users\james\AppData\Local\Temp\ipykernel_14052\3673215081.py:14: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


Loaded DataFrame for table: raw_player_defense_df
Loaded DataFrame for table: raw_player_gca_df
Loaded DataFrame for table: raw_player_keepersadv_df
Loaded DataFrame for table: raw_player_passing_df
Loaded DataFrame for table: raw_player_passing_type_df
Loaded DataFrame for table: raw_player_possession_df
Loaded DataFrame for table: raw_player_shooting_df
Loaded DataFrame for table: raw_stats_player_standard_df


In [10]:
dfs.keys()
#dfs["player_defense_df"].head()

dict_keys(['raw_player_defense_df', 'raw_player_gca_df', 'raw_player_keepersadv_df', 'raw_player_passing_df', 'raw_player_passing_type_df', 'raw_player_possession_df', 'raw_player_shooting_df', 'raw_stats_player_standard_df'])

In [11]:
import hashlib

def make_id(row):
    unique_str = f"{row['Player']}_{row['Nation']}_{row['Born']}"
    return hashlib.md5(unique_str.encode()).hexdigest()

for name, df in dfs.items():
    df['id'] = df.apply(make_id, axis=1)
    dfs[name] = df

In [ ]:
drop_rules = {
    'eredivisie': list(range(2010, 2018)),
    'championship': list(range(2010, 2018)),
    'primeira liga': list(range(2010, 2018)),
    'bundesliga': list(range(2010, 2017)),
    'premier league': list(range(2010, 2017)),
    'la liga': list(range(2010, 2017)),
    'serie a': list(range(2010, 2017)),
    'ligue 1': list(range(2010, 2017)),
    'scottish premiership': list(range(2010, 2026))
}

for key, df in dfs.items():
    if 'League' not in df.columns:
        continue

    df[['league_name', 'season']] = df['League'].str.rsplit(' ', n=1, expand=True)
    df['league_name'] = df['league_name'].str.strip().str.lower()
    df['Player'] = df['Player'].str.strip().str.lower()
    df['Nation'] = df['Nation'].str.strip().str.lower()
    df['Squad'] = df['Squad'].str.strip().str.lower()
    df['League'] = df['League'].str.strip().str.lower()
    df['season_start'] = pd.to_numeric(df['season'].str[:4], errors='coerce')
    df = df.dropna(subset=['season_start'])
    df['season_start'] = df['season_start'].astype(int)

    mask = (df['league_name'].isin(drop_rules.keys())) & \
           df.apply(lambda row: row['season_start'] in drop_rules[row['league_name']], axis=1)
    dfs[key] = df[~mask]

In [40]:
dfs["raw_player_defense_df"].tail()

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Tkl,TklW,...,Int,Tkl+Int,Clr,Err,Matches,League,id,league_name,season,season_start
39182,550,mehdi zeffane,dz alg,"DF,MF",rennes,25,1992,11.0,17,12,...,12,29,44,0,Matches,ligue 1 2017-2018,2e834c7c9543d32c531d6f2891767cd7,ligue 1,2017-2018,2017
39183,Rk,player,nation,Pos,squad,Age,Born,90s,Tkl,TklW,...,Int,Tkl+Int,Clr,Err,Matches,ligue 1 2017-2018,168d351c2bb7728c687026e8b45ce821,ligue 1,2017-2018,2017
39184,551,erwin zelazny,fr fra,GK,troyes,25,1991,17.0,1,1,...,2,3,10,1,Matches,ligue 1 2017-2018,35a8fe9f3afbf85f49877043a979c2dc,ligue 1,2017-2018,2017
39185,552,kévin zohi,ml mli,MF,strasbourg,20,1996,0.0,0,0,...,0,0,0,0,Matches,ligue 1 2017-2018,16fb71d35478f3c58e8183ea5b6286f9,ligue 1,2017-2018,2017
39186,553,bongani zungu,za rsa,MF,amiens,24,1992,23.2,54,37,...,38,92,41,2,Matches,ligue 1 2017-2018,dedb94bf18fcb1b8db8432fdfc4a13d4,ligue 1,2017-2018,2017
